# Regression with Assymmetric loss function

1. Feature extraction --> windowing(3 mins), handcrafted features, 
2. Data split
3. Conduct regression with different loss functions
4. Scoring methods :
    MAE
    MBR
    MSE
5. Get results for all composite, ziptie and combined
6. Include Gender as the feature

In [39]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os
import sys
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import math
import seaborn as sns

## Scikit related
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, chi2, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDClassifier
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from scipy import signal
from scipy import integrate
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, KernelPCA
from sklearn.model_selection import cross_val_score
from scipy.stats import skew
from scipy.stats import norm, kurtosis
from sklearn.manifold import TSNE
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.neighbors import KernelDensity

# Updated helper fucntion made for the new data from formal user study

from helper_functions_user_study_2_0 import get_all_file_paths
from helper_functions_user_study_2_0 import regr_feature_extract_one_segment
from helper_functions_user_study_2_0 import feature_scaling
from helper_functions_user_study_2_0 import regr_segment_time_series
# from helper_functions_user_study import SVR_linear
# from helper_functions_user_study import SVR_poly
# from helper_functions_user_study import SVR_rbf
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Prepare Dataframe for Regression Tasks

In [40]:
def __read_csv_drop_unnamed__(csv_path) :
    pd_test = pd.read_csv(csv_path)
    pd_test.columns.str.match("Unnamed")
    pd_test = pd_test.loc[:,~pd_test.columns.str.match("Unnamed")]
    return pd_test


In [41]:
def conv_segment_df (segment) :
    df = pd.DataFrame(segment, columns=[
        ['Time',
 'HR_Processed',
 'HR',
 'HRV',
 'RR',
 'RRSQI',
 'ECGSQI',
 'ECG',
 'acc_X',
 'acc_Y',
 'acc_Z',
 'Temperature',
 'IMU_1_ax_g_',
 'IMU_1_ay_g_',
 'IMU_1_az_g_',
 'IMU_1_gx_dps_',
 'IMU_1_gy_dps_',
 'IMU_1_gz_dps_',
 'IMU_2_ax_g_',
 'IMU_2_ay_g_',
 'IMU_2_az_g_',
 'IMU_2_gx_dps_',
 'IMU_2_gy_dps_',
 'IMU_2_gz_dps_',
 'IMU_3_ax_g_',
 'IMU_3_ay_g_',
 'IMU_3_az_g_',
 'IMU_3_gx_dps_',
 'IMU_3_gy_dps_',
 'IMU_3_gz_dps_',
 'IMU_4_ax_g_',
 'IMU_4_ay_g_',
 'IMU_4_az_g_',
 'IMU_4_gx_dps_',
 'IMU_4_gy_dps_',
 'IMU_4_gz_dps_',
 'IMU_5_ax_g_',
 'IMU_5_ay_g_',
 'IMU_5_az_g_',
 'IMU_5_gx_dps_',
 'IMU_5_gy_dps_',
 'IMU_5_gz_dps_',
 'Physical Fatigue - Initial',
 'Physical Fatigue - Final',
 'Mental Fatigue - Initial',
 'Mental Fatigue - Final',
 'Performance rating',
 'Age',
 'Weight',
 'Height',
 'Weights added',
 'Gender']
    ])
    return df  
def file_segment_features(test_seg) :
#    print('Inside segment features') 
   # Give segmented 3D array to this --> Features dataframe
   test_one_seg = conv_segment_df(test_seg[0])
   features_df = regr_feature_extract_one_segment(test_one_seg)
   #features_df.head()
   for i in range(1, np.shape(test_seg)[0]) :
    # print('===============================================================I am in segment ', i)
    temp_seg = conv_segment_df(test_seg[i])
    df = regr_feature_extract_one_segment(temp_seg)
    features_df = features_df.append(df)
    
   return features_df 

In [42]:
# Get all Composite Files
window_size = 360 ## 120 is one min
source_data_path_composite = 'MxD_Data_User_Study/composite/'
# 'composite_train/'
source_data_path_ziptie = 'MxD_Data_User_Study/ziptie/'
# 'ziptie_train/'


# this function is to get all the interpolated and segmented data
folder_list_composite = os.listdir(source_data_path_composite)
composite_file_names = get_all_file_paths(source_data_path_composite, folder_list_composite)

composite_main_dataframe = pd.DataFrame(__read_csv_drop_unnamed__(composite_file_names[0]))
test_seg_0 = regr_segment_time_series(window_size, composite_main_dataframe)
composite_main_dataframe = file_segment_features(test_seg_0)
for i in range(1,len(composite_file_names)):
  data = __read_csv_drop_unnamed__(composite_file_names[i])
  if (len(data) < window_size) :
    pass
  else :
    test_seg = regr_segment_time_series(window_size,data)
    features_df = file_segment_features(test_seg)
    composite_main_dataframe = composite_main_dataframe.append(features_df)


AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
# this function is to get all the interpolated and segmented data
folder_list_ziptie = os.listdir(source_data_path_ziptie)
ziptie_file_names = get_all_file_paths(source_data_path_ziptie, folder_list_ziptie)

ziptie_main_dataframe = pd.DataFrame(__read_csv_drop_unnamed__(ziptie_file_names[0]))
test_seg_0 = regr_segment_time_series(window_size, ziptie_main_dataframe)
ziptie_main_dataframe = file_segment_features(test_seg_0)
for i in range(1,len(ziptie_file_names)):
  data = __read_csv_drop_unnamed__(ziptie_file_names[i])
  if (len(data) < window_size) :
    pass
  else :
    test_seg = regr_segment_time_series(window_size,data)
    features_df = file_segment_features(test_seg)
    ziptie_main_dataframe = ziptie_main_dataframe.append(features_df)

## Drop rows with 0 HR

In [ ]:
composite_main_dataframe = composite_main_dataframe.loc[~((composite_main_dataframe['max_hr'] == 0))]

In [ ]:
ziptie_main_dataframe = ziptie_main_dataframe.loc[~((ziptie_main_dataframe['max_hr'] == 0))]

In [ ]:
combined_dataframe = ziptie_main_dataframe
combined_dataframe = combined_dataframe.append(composite_main_dataframe)

In [ ]:
## Drop the gyro features
composite_main_dataframe = composite_main_dataframe.drop(['range_gyro_imu1', 'rms_imu1_gyro',
                                                          'range_gyro_imu2', 'rms_imu2_gyro',
                                                          'range_gyro_imu3', 'rms_imu3_gyro',
                                                          'range_gyro_imu4', 'rms_imu4_gyro',
                                                           'rms_imu5_gyro'], axis='columns')
ziptie_main_dataframe = ziptie_main_dataframe.drop(['range_gyro_imu1', 'rms_imu1_gyro',
                                                          'range_gyro_imu2', 'rms_imu2_gyro',
                                                          'range_gyro_imu3', 'rms_imu3_gyro',
                                                          'range_gyro_imu4', 'rms_imu4_gyro',
                                                           'rms_imu5_gyro'], axis='columns')                                                           
combined_dataframe = combined_dataframe.drop(['range_gyro_imu1', 'rms_imu1_gyro',
                                                          'range_gyro_imu2', 'rms_imu2_gyro',
                                                          'range_gyro_imu3', 'rms_imu3_gyro',
                                                          'range_gyro_imu4', 'rms_imu4_gyro',
                                                           'rms_imu5_gyro'], axis='columns') 


# composite_main_dataframe_test = composite_main_dataframe_test.drop(['range_gyro_imu1', 'rms_imu1_gyro',
#                                                           'range_gyro_imu2', 'rms_imu2_gyro',
#                                                           'range_gyro_imu3', 'rms_imu3_gyro',
#                                                           'range_gyro_imu4', 'rms_imu4_gyro',
#                                                            'rms_imu5_gyro'], axis='columns')
# ziptie_main_dataframe_test = ziptie_main_dataframe_test.drop(['range_gyro_imu1', 'rms_imu1_gyro',
#                                                           'range_gyro_imu2', 'rms_imu2_gyro',
#                                                           'range_gyro_imu3', 'rms_imu3_gyro',
#                                                           'range_gyro_imu4', 'rms_imu4_gyro',
#                                                            'rms_imu5_gyro'], axis='columns')                                                           
# combined_dataframe_test = combined_dataframe_test.drop(['range_gyro_imu1', 'rms_imu1_gyro',
#                                                           'range_gyro_imu2', 'rms_imu2_gyro',
#                                                           'range_gyro_imu3', 'rms_imu3_gyro',
#                                                           'range_gyro_imu4', 'rms_imu4_gyro',
#                                                            'rms_imu5_gyro'], axis='columns') 

# Get ready for training

In [ ]:
## Scale the dataset
df_scaled_combined = feature_scaling(0,10, combined_dataframe)
df_scaled_composite = feature_scaling(0,10, composite_main_dataframe)
df_scaled_ziptie = feature_scaling(0,10, ziptie_main_dataframe)

In [ ]:
ziptie_main_dataframe.head()

## Training only Composite Task

In [ ]:
## Drop initial fatigue for training
""" July 11th :Does it make sense to even include the initial fatigue anymore? --> It would be like including the time delayed version of our target itself.
First trial without any initial fatigue. Include initial fatigue after you increase the window size.
"""
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(
    df_scaled_composite.drop(['final_fatigue', 'init_fatigue'], axis='columns'), composite_main_dataframe['final_fatigue'], test_size=0.3)

X_train = X_train.to_numpy()
Y_train = Y_train.to_numpy()

X_test = X_test.to_numpy()
Y_test = Y_test.to_numpy()

## Assymetric losses

In [ ]:
E = np.linspace(-1,1, 1000)
LINEX_loss = lambda E, a: (2/a**2)*(np.exp(a*E) - a*E - 1)
plt.plot(E, LINEX_loss(E, a= -1))
plt.xlabel('Error (Y_pred-Y_actual)')
plt.ylabel('Loss')

In [ ]:
ASSYM_loss = lambda E : np.where((E)<0, 2*(E**2), (E**2))
plt.plot(E, ASSYM_loss(E))
plt.xlabel('Error (Y_pred-Y_actual)')
plt.ylabel('Loss')

In [ ]:
def __compute_mae__(y_pred, y_test) :
    ## traslatet to an array
    y_pred = np.array(y_pred)
    y_test = np.array(y_test)
    error = np.zeros(len(y_test))
    error = np.array(error)
    neg_error = 0
    pos_error = 0
    mae = np.mean(np.abs(y_pred - y_test))
    print('MAE = ', mae)


def __compute_rel_predictions__(y_pred, y_test) :
    ## traslatet to an array
    y_pred = np.array(y_pred)
    y_test = np.array(y_test)
    error = np.zeros(len(y_test))
    error = np.array(error)
    neg_error = 0
    pos_error = 0
    for i in range(0, len(y_test)) :
        error[i] = y_pred[i] - y_test[i]
        if error[i] < 0 :
            neg_error += 1 ## Under prediction --> more dangerous
        elif error[i] > 0 :
            pos_error += 1 ## Over prediction
    print('Underpredictions :', neg_error/len(y_test)) 
    print('Overpredictions :', pos_error/len(y_test)) 

In [ ]:
"""Sources : 
1. https://arxiv.org/pdf/1603.02754.pdf 
2. https://github.com/froukje/articles/blob/main/02_custom_loss_xgboost.ipynb
3. https://www.datatrigger.org/post/asymmetric_loss/ 
"""
# def mean_bias_error(y_pred, y_true) :

def xgb_model(X_train, y_train,X_val, y_val, 
              objective='reg:squarederror',
              learning_rate=0.3,
              min_child_weight=1,
              lambda_=1,
              gamma=0, model_ckhpt_path='model_xgboost.json'):
    
    # Initialize XGB with objective function
    parameters = {"objective": objective,
              "n_estimators": 200,
              "eta": learning_rate,
              "lambda": lambda_,
              "gamma": gamma,
              "max_depth": None,
              "min_child_weight": min_child_weight,
              "verbosity": 0}

    
    model = xgb.XGBRegressor(**parameters)
    model.fit(X_train, y_train)
    model.save_model(model_ckhpt_path)
     
    # generate predictions
    y_pred_train = model.predict(X_train).reshape(-1,1)
    y_pred = model.predict(X_val).reshape(-1,1)
    
    # calculate errors
    rmse_train = mean_squared_error(y_pred_train, y_train, squared=False)
    rmse_val = mean_squared_error(y_pred, y_val, squared=False)
    print(f"rmse training: {rmse_train:.3f}\t rmse validation: {rmse_val:.3f}")
    __compute_rel_predictions__(y_pred, y_val)
    __compute_mae__(y_pred, y_val)
    
    # plot results
    y_train = np.array(y_train).reshape(-1,1)
    y_val = np.array(y_val).reshape(-1,1)
    
    # y_train_dummy = y_train
    fig, axes = plt.subplots(1, 2, figsize=(15,5))
    axes[0].scatter(y_pred_train, y_train, alpha=0.5, s=10)
    axes[0].plot(y_train, y_train, 'r')
    axes[0].set_xlabel('predicted values')
    axes[0].set_ylabel('true values')
    axes[0].set_title(f"Training, rmse: {rmse_train:.3f}")
    axes[1].scatter(y_pred, y_val, alpha=0.5, s=10)
    axes[1].plot(y_val, y_val, 'r')
    axes[1].set_xlabel('predicted values')
    axes[1].set_ylabel('true values')
    axes[1].set_title(f"Validation, rmse: {rmse_val:.3f}")
    
    fig, axes = plt.subplots(1, 2, figsize=(15,5))
    frequency, bins = np.histogram(y_train, bins=50, range=[np.min(y_pred_train), np.max(y_pred_train)])
    axes[0].hist(y_train, alpha=0.5, bins=bins, density='true', label="train")
    axes[0].hist(y_pred_train, alpha=0.5, bins=bins, density='true', label="predictions")
    axes[0].set_xlabel('Label Scale (0-10)')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title(f"Histogram of label distribution")
    axes[0].legend()
    axes[1].hist(y_val, alpha=0.5, bins=bins, density='true', label="validation")
    axes[1].hist(y_pred, alpha=0.5, bins=bins, density='true', label="prediction")
    axes[1].set_xlabel('Label Scale (0-10)')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title(f"Histogram of label distribution")
    axes[1].legend()
    return y_pred_train, y_pred

In [ ]:
def assym_loss_over_pred(y_val, y_pred):
    grad = np.where((y_val - y_pred)>0, -2.0*2.0*(y_val - y_pred), -2.0*(y_val - y_pred))
    hess = np.where((y_val - y_pred)>0, 2.0*2.0, 2.0)
    return grad, hess

def assym_loss_under_pred(y_val, y_pred):
    grad = np.where((y_val - y_pred)<0, -2.0*(y_val - y_pred), -2.0*2.0*(y_val - y_pred))
    hess = np.where((y_val - y_pred)<0, 2.0, 2.0*2.0)
    return grad, hess    

In [ ]:
def __compute_mbe__(y_pred, y_test):
    mbe = np.zeros(len(y_test))
    for i in range(0, len(y_test)) :
        mbe[i] = y_pred[i] - y_test[i]
    print('MBE = ', np.mean(mbe))    
    return mbe   

In [ ]:
def __compute_rel_predictions__(y_pred, y_test) :
    error = np.zeros(len(y_test))
    neg_error = 0
    pos_error = 0
    for i in range(0, len(y_test)) :
        error[i] = y_pred[i] - y_test[i]
        if error[i] < 0 :
            neg_error += 1 ## Under prediction --> more dangerous
        elif error[i] > 0 :
            pos_error += 1 ## Over prediction
    print('Underpredictions :', neg_error/len(y_test)) 
    print('Overpredictions :', pos_error/len(y_test)) 

In [ ]:
def LinexLoss_over_pred(y_predicted, y_true):
    a = -0.5
    E = y_true - y_predicted
    grad = (2/a)*(np.exp(a*E) - 1)## negative of first derivative || Remember you are taking the derivative wrt y_pred
    hess = 2*(np.exp(a*E))
    # grad = 4*E**3
    # hess = 12*E**2
    return grad, hess

def LinexLoss_under_pred(y_predicted, y_true):
    a = 1
    E = y_true - y_predicted
    grad = (2/a)*(np.exp(a*E) - 1)## negative of first derivative || Remember you are taking the derivative wrt y_pred
    hess = 2*(np.exp(a*E))
    # grad = 4*E**3
    # hess = 12*E**2
    return grad, hess    

In [ ]:
_, y_pred_over_assym = xgb_model(X_train, Y_train, X_test, Y_test, objective=assym_loss_over_pred, learning_rate=0.01, gamma=10)
## higher the gamma --> more regularization

In [ ]:
_, y_pred_over_assym = xgb_model(X_train, Y_train, X_test, Y_test, objective=LinexLoss_over_pred, learning_rate=0.01, gamma=10)

## Train ziptie

In [ ]:

X_train_zp, X_test_zp, Y_train_zp, Y_test_zp = sklearn.model_selection.train_test_split(
    df_scaled_ziptie.drop(['final_fatigue', 'init_fatigue'], axis='columns'), ziptie_main_dataframe['final_fatigue'], test_size=0.3)

X_train_zp = X_train_zp.to_numpy()
Y_train_zp = Y_train_zp.to_numpy()

X_test_zp  = X_test_zp.to_numpy()
Y_test_zp  = Y_test_zp.to_numpy()

In [ ]:
_, y_pred_over_assym_zp = xgb_model(X_train_zp, Y_train_zp, X_test_zp, Y_test_zp, objective=assym_loss_over_pred, learning_rate=0.05, gamma=20)

In [ ]:
_, y_pred_over_linex_zp = xgb_model(X_train_zp, Y_train_zp, X_test_zp, Y_test_zp, objective=LinexLoss_over_pred, learning_rate=0.05, gamma=20)

## Combined Tasks

In [ ]:
X_train_combo, X_test_combo, Y_train_combo, Y_test_combo = sklearn.model_selection.train_test_split(
    df_scaled_combined.drop(['final_fatigue', 'init_fatigue'], axis='columns'), df_scaled_combined['final_fatigue'], test_size=0.3)

X_train_combo = X_train_combo.to_numpy()
Y_train_combo = Y_train_combo.to_numpy()

X_test_combo  = X_test_combo.to_numpy()
Y_test_combo  = Y_test_combo.to_numpy()

In [ ]:
np.random.seed(121212)
_, y_pred_over_linex_combo = xgb_model(X_train_combo, Y_train_combo, X_test_combo, Y_test_combo, objective=LinexLoss_over_pred, learning_rate=0.01, gamma=30)

In [ ]:
np.random.seed(2711)
_, y_pred_over_assym_combo = xgb_model(X_train_combo, Y_train_combo, X_test_combo, Y_test_combo, objective=assym_loss_over_pred, learning_rate=0.05, gamma=10)